In [1]:

from hybrid_recommenders.item_knn_cf_cbf_weighted import item_knn_cf_cbf_weighted
from utils.evaluator.evaluator import evaluator
from utils.submission_helper import submission_helper
from utils.data_manager.data_manager import data_manager
from utils.data_manager.splitter import splitter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
training, validation, test = splitter().get_train_evaluation_test(percentage_of_training_data=0.6,
                                                                  percentage_of_validation_data=0.2)

In [3]:
weights1 = [i for i in np.arange(0.05, 1, 0.05)]
print("weights 1: ", weights1)
print(weights1)
weights2 = list(reversed([i for i in np.arange(0.05, 1, 0.05)]))
print("weights2: ", weights2)
weights = list(zip(weights1,weights2))
weights=[list(elem ) for elem in weights]
weights = [i for i in weights if i[0]!=i[1]]
weights.append([0.5, 0.5])
print(weights)

weights 1:  [0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.3, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.6500000000000001, 0.7000000000000001, 0.7500000000000001, 0.8, 0.8500000000000001, 0.9000000000000001, 0.9500000000000001]
[0.05, 0.1, 0.15000000000000002, 0.2, 0.25, 0.3, 0.35000000000000003, 0.4, 0.45, 0.5, 0.55, 0.6000000000000001, 0.6500000000000001, 0.7000000000000001, 0.7500000000000001, 0.8, 0.8500000000000001, 0.9000000000000001, 0.9500000000000001]
weights2:  [0.9500000000000001, 0.9000000000000001, 0.8500000000000001, 0.8, 0.7500000000000001, 0.7000000000000001, 0.6500000000000001, 0.6000000000000001, 0.55, 0.5, 0.45, 0.4, 0.35000000000000003, 0.3, 0.25, 0.2, 0.15000000000000002, 0.1, 0.05]
[[0.05, 0.9500000000000001], [0.1, 0.9000000000000001], [0.15000000000000002, 0.8500000000000001], [0.2, 0.8], [0.25, 0.7500000000000001], [0.3, 0.7000000000000001], [0.35000000000000003, 0.6500000000000001], [0.4, 0.6000000000000001], [0.45, 0.55], [0.55, 0.45], [0

In [4]:
map_vector = []
cf_similarity = 'asymmetric'
cbf_similarity = 'jaccard'
k_couple = weights[0]
recommender = item_knn_cf_cbf_weighted(weight_cf=k_couple[0],
                                        weight_cbf=k_couple[1],
                                        cf_similarity=cf_similarity,
                                        cbf_similarity=cbf_similarity)
recommender.fit(training_set=training,
                    k_cf=310,
                    k_cbf=90,
                    shrink_cf=246,
                    shrink_cbf=36)
map_vector.append(evaluator.evaluate_already_trained(recommender=recommender)[2])
print("first evaluation: ", map_vector[-1])
for k_couple in weights[1:]:
    recommender.set_weights(k_couple[0], k_couple[1])
    map_ = evaluator.evaluate_already_trained(recommender=recommender)[2]
    print(map_)
    map_vector.append(map_)
map_vector

Similarity column 25975 ( 100 % ), 3014.70 column/sec, elapsed time 0.14 min
Similarity column 25975 ( 100 % ), 1393.28 column/sec, elapsed time 0.31 min
first evaluation:  0.022030330870803614
0.022372282653339102
0.02339985903331212
0.024941610624197214
0.026285505884899644
0.027729029353835866
0.029199123077368216
0.030609068182515155
0.03168948998751933
0.0339445921805636
0.034857030655372254
0.036177730511833664
0.03614140305988955
0.03638561361701767
0.03605312097610516
0.03488644110879144
0.03300996599121965
0.030696830067672935
0.033109279202905434


[0.022030330870803614,
 0.022372282653339102,
 0.02339985903331212,
 0.024941610624197214,
 0.026285505884899644,
 0.027729029353835866,
 0.029199123077368216,
 0.030609068182515155,
 0.03168948998751933,
 0.0339445921805636,
 0.034857030655372254,
 0.036177730511833664,
 0.03614140305988955,
 0.03638561361701767,
 0.03605312097610516,
 0.03488644110879144,
 0.03300996599121965,
 0.030696830067672935,
 0.033109279202905434]

In [5]:
chosen_weights = weights[np.argmax([map_vector])]
chosen_weights

[0.7500000000000001, 0.25]

In [6]:
np.max(map_vector)


0.03638561361701767

In [4]:
submission_recommender = item_knn_cf_cbf_weighted(weight_cf=0.5,
                                                  weight_cbf=0.5,
                                                  cf_similarity='asymmetric',
                                                  cbf_similarity='jaccard')
submission_recommender.fit(training_set=training.copy(),
                           k_cf=310,
                           k_cbf=90,
                           shrink_cf=246,
                           shrink_cbf=36)
map_ = evaluator.evaluate_already_trained(recommender=submission_recommender)[2]
map_

Similarity column 25975 ( 100 % ), 2478.98 column/sec, elapsed time 0.17 min
Similarity column 25975 ( 100 % ), 1023.04 column/sec, elapsed time 0.42 min


0.03638561361701767